In [11]:
# -*- coding: utf-8 -*-
"""
- Leer los proyectos desde el CSV "proyecto_limpio_4".
- Ejecutar el modelo fila por fila (saltando filas con ID = 0).
- Guardar la respuesta JSON en la columna "JSON_RPTA".
- Exportar el resultado a "resultados1.csv".
"""

!pip install -q google-generativeai

In [12]:

import google.generativeai as genai
from google.colab import userdata
import os
import json
import time
import pandas as pd  # <- NUEVO: para manejar el CSV

# -------------------------------------------------------------------
# 0. Configurar API de Gemini
# -------------------------------------------------------------------
API_KEY = userdata.get('BRECHAS_API_KEY')

if API_KEY is None:
    raise ValueError(
        "No se encontró la API Key. Por favor, guárdala en los secretos de Colab "
        "bajo el nombre 'BRECHAS_API_KEY'."
    )

genai.configure(api_key=API_KEY)
print("API de Gemini configurada exitosamente.")

# Elegir modelo
GEMINI_MODEL_NAME = "gemini-2.5-flash"
model = genai.GenerativeModel(GEMINI_MODEL_NAME)

API de Gemini configurada exitosamente.


##1. Definiciones de categorías

In [13]:
# -------------------------------------------------------------------
# 1. Definiciones de categorías
# -------------------------------------------------------------------
CATEGORIAS = {
    "CATEGORIA_1": {
        "id": 1,
        "nombre": "servicio de alcantarillado u otras formas de disposicion sanitaria de excretas",
        "definicion": """
Texto describe al menos uno (1) de los siguientes servicios:

1. SERVICIO DE ALCANTARILLADO: incluye al menos uno (1) de los siguientes procesos con aguas residuales:
   recolección, impulsión, conducción hasta el punto de entrega y tratamiento.

2. SERVICIO DE DISPOSICIÓN SANITARIA DE EXCRETAS: incluye instalaciones intradomiciliarias
   (como Unidades Básicas Sanitarias - UBS, letrinas, etc.) y/o procesos para la disposición final del agua residual
   o las excretas.
"""
    },

    "CATEGORIA_2": {
        "id": 2,
        "nombre": "servicio de agua potable mediante red publica o pileta publica",
        "definicion": """
comprende al menos uno (1) de los siguientes servicios:
1. sistema de producción: captación, almacenamiento y conducción de agua cruda, tratamiento y conducción de agua potable.
2. sistema de distribución: almacenamiento, distribución, entrega y medición al usuario.

comprende instalaciones como:
1. red pública: conexión domiciliaria.
2. pileta públicas: punto de abastecimiento de agua potable instalado en el espacio público.
"""
    },

    "CATEGORIA_3": {
        "id": 3,
        "nombre": "aguas residuales no tratadas",
        "definicion": """
El servicio de tratamiento de aguas residuales para disposición final incluye instalar, mejorar o ampliar instalaciones que permitan procesos con aguas residuales.

PROCESOS: el servicio comprende uno o ninguno:
* recolección
* impulsión o conducción hasta la planta de tratamiento
* tratamiento o reúso del efluente
* disposición final

EJEMPLOS DE TECNOLOGÍAS DE TRATAMIENTO INCLUIDAS (no limitativo):
* PTAR – Plantas de Tratamiento de Aguas Residuales.
* Biorreactores de membrana (MBR).
* Ósmosis inversa y ultrafiltración.
"""
    },

    "CATEGORIA_4": {
        "id": 4,
        "nombre": "m2 de espacios publicos verdes por implementar",
        "definicion": """
Incluye espacios públicos como los siguientes (no limitativo):
* Parque
* Plaza
* Jardín
* Alameda
* Bosque Urbano
* Zona de Esparcimiento
"""
    },

    "CATEGORIA_5": {
        "id": 5,
        "nombre": "servicios de movilidad a traves de pistas y veredas",
        "definicion": """
El SERVICIO DE MOVILIDAD permite la transitabilidad a través de infraestructura vial.

Ejemplos de infraestructura vial urbana (no limitativo):
* pista
* calle
* pasaje
* avenida
* vías urbanas
* vías principales
* vereda
* puente
* óvalo
"""
    },

    "CATEGORIA_6": {
        "id": 6,
        "nombre": "servicio de educacion inicial",
        "definicion": """
El SERVICIO DE EDUCACIÓN INICIAL atiende a niños menores de 6 años con enfoque intercultural e inclusivo.
Promueve el desarrollo infantil en dimensiones cognitiva, física, motora, social y emocional.

Incluye la implementación de recursos educativos:
* infraestructura
* equipos
* personal
* organización
* capacidad de gestión
* otros

Servicios complementarios (uno o ninguno):
* atención en salud
* nutrición
* protección
* acceso al registro legal de identidad
* servicios de cuidado
* otros servicios que aseguren condiciones básicas para el desarrollo infantil
"""
    },

    "CATEGORIA_7": {
        "id": 7,
        "nombre": "servicio de educacion primaria",
        "definicion": """
El SERVICIO DE EDUCACIÓN PRIMARIA atiende a niños y niñas desde los 6 años.

Incluye:
* infraestructura
* equipos
* personal
* organización
* capacidad de gestión

Objetivos del servicio:
* desarrollo del pensamiento lógico y matemático
* comunicación, expresión artística y psicomotricidad
* aprendizajes en ciencia, humanidades y tecnología
* estrategias diversificadas según ritmos y niveles de aprendizaje
* atención a la pluralidad lingüística y cultural
"""
    },

    "CATEGORIA_8": {
        "id": 8,
        "nombre": "servicio de educacion secundaria",
        "definicion": """
El SERVICIO DE EDUCACIÓN SECUNDARIA atiende a adolescentes e incluye:

* infraestructura
* equipos
* personal
* organización
* capacidad de gestión

Objetivos del servicio:
* formación humanística, científica y tecnológica
* capacitación para el trabajo
* promoción de competencias emprendedoras orientadas al desarrollo de proyectos productivos con uso de tecnologías

NO se considera parte del servicio:
* Colegios de Alto Rendimiento (COAR)

Nota: II.EE = Instituciones Educativas.
"""
    }
}

## 2. Construcción del prompt

In [14]:

# -------------------------------------------------------------------
# 2. Construcción del prompt
# -------------------------------------------------------------------
def construir_prompt(texto_proyecto: str) -> str:
    """
    Construye un prompt en español para que Gemini actúe como
    clasificador multi-etiqueta basado en las definiciones de categorías.

    Recibe directamente el título/descripción del proyecto como string.
    """
    texto_proyecto = str(texto_proyecto).strip()

    partes_categorias = []
    for clave, info in CATEGORIAS.items():
        partes_categorias.append(
            f"- ID: {info['id']}\n"
            f"  NOMBRE: {info['nombre']}\n"
            f"  DEFINICION: {info['definicion'].strip()}\n"
        )
    texto_categorias = "\n".join(partes_categorias)

    prompt = f"""
Eres un modelo de lenguaje experto en clasificación de títulos de proyectos públicos
según brechas de infraestructura y servicios definidas por el SNPMGI del Perú.

Tu única tarea es leer el título del proyecto y asignarle UNA O VARIAS categorías
de servicios publicos de la lista definida abajo.

CATEGORÍAS DISPONIBLES:
{texto_categorias}

REGLAS ESTRICTAS:
- Analiza el significado del título del proyecto, no solo palabras sueltas.
- Asigna múltiples categorías solo si el título realmente cubre más de una brecha.
- No inventes información adicional que no esté presente o inferida razonablemente del título del proyecto.

FORMATO DE RESPUESTA (OBLIGATORIO):
Responde ÚNICAMENTE con JSON válido, sin texto adicional, sin explicaciones, sin backticks.
Ejemplo de formato:

{{
  "labels": [
    {{
      "label": "NOMBRE_DE_CATEGORIA_1",
      "id":1,
      "confianza": 0.95,
      "justificacion": "Texto de la justificacion"
    }},
    {{
      "label": "NOMBRE_DE_CATEGORIA_2",
      "id":3,
      "confianza": 0.98,
      "justificacion": "Texto de la justificacion"
    }}
  ]
}}

donde:
- "labels" es una lista de objetos.
- "label" es el nombre de la categoría seleccionada.
- "id": es el identificador numérico único asignado a cada categoría. Debes devolver exactamente el id asociado a la categoría, según la lista de categorías proporcionada en el prompt.
- "confianza": representa el nivel de certeza del modelo sobre la asignación de una categoría. Debe ser un valor numérico entre 0 y 1.
- "justificacion": explica por qué el proyecto fue clasificado en esa categoria usando unicamente la DEFINICIÓN de la categoria. máximo 200 palabras.

Si el título del proyecto es ambiguo o no coincide con ninguna definición, debes devolver:

{{
  "labels": [
    {{
      "label": "NO_CLASIFICADO",
      "id": 0,
      "confianza": 0.0,
      "justificacion": "El texto no es suficiente o no coincide con ninguna categoría."
    }}
  ]
}}

TEXTO A CLASIFICAR:
\"\"\"{texto_proyecto}\"\"\"
"""
    return prompt

3. Utilidad para limpiar y extraer JSON de la respuesta del modelo


In [15]:
# -------------------------------------------------------------------
# 3. Utilidad para limpiar y extraer JSON de la respuesta del modelo
# -------------------------------------------------------------------
def extraer_json_de_texto(texto: str) -> str:
    """
    Intenta limpiar la salida del modelo y extraer solo la parte JSON.

    - Elimina backticks de bloques de código (```json ... ```).
    - Toma el contenido entre la primera '{' y la última '}'.
    - Valida que sea JSON.
    """
    if texto is None:
        raise json.JSONDecodeError("Respuesta vacía del modelo", "", 0)

    s = texto.strip()

    # 1) Quitar bloques de código markdown si los hubiera
    if s.startswith("```"):
        # eliminar primera línea (``` o ```json)
        first_newline = s.find("\n")
        if first_newline != -1:
            s = s[first_newline + 1:]
        # eliminar ``` final si está
        if s.endswith("```"):
            s = s[:-3]
        s = s.strip()

    # 2) Quedarnos con lo que está entre la primera '{' y la última '}'
    start = s.find("{")
    end = s.rfind("}")
    if start != -1 and end != -1 and end > start:
        s = s[start:end + 1]

    # 3) Intentar parsear como JSON
    obj = json.loads(s)  # si falla, lanza JSONDecodeError
    # 4) Devolverlo normalizado (string JSON)
    return json.dumps(obj, ensure_ascii=False)

 4. Función para llamar a Gemini y devolver JSON limpio

In [16]:


# -------------------------------------------------------------------
# 4. Función para llamar a Gemini y devolver JSON limpio
# -------------------------------------------------------------------
def ejecutar_prompt(prompt: str, max_reintentos: int = 3, espera_segundos: int = 2) -> str:
    """
    Envía el prompt al modelo de Gemini y devuelve la respuesta como JSON limpio (string).
    Si la respuesta no es JSON válido, devuelve un JSON de error.
    """
    for intento in range(1, max_reintentos + 1):
        try:
            respuesta = model.generate_content(prompt)
            texto = (respuesta.text or "").strip()

            try:
                json_limpio = extraer_json_de_texto(texto)
                return json_limpio
            except json.JSONDecodeError as e:
                # Si el modelo no respetó el formato, devolvemos error pero guardamos la respuesta cruda
                return json.dumps({
                    "labels": [],
                    "error": "La respuesta del modelo no es JSON válido",
                    "detalle_error": str(e),
                    "raw_response": texto
                }, ensure_ascii=False)

        except Exception as e:
            print(f"[Intento {intento}] Error al llamar a Gemini: {e}")
            time.sleep(espera_segundos)

    # Si falla todos los intentos, devolvemos un JSON de error
    return json.dumps({
        "labels": [],
        "error": "No se pudo obtener respuesta del modelo luego de varios intentos."
    }, ensure_ascii=False)

5. Punto de entrada principal: leer CSV y procesar fila por fila

In [17]:
# -------------------------------------------------------------------
# 5. Punto de entrada principal: leer CSV y procesar fila por fila
# -------------------------------------------------------------------
if __name__ == "__main__":
    print("=== Clasificador de proyectos públicos (Gemini) ===")
    print("Leyendo archivo 'proyecto_limpio_4.csv'...")

    # Ajusta 'sep' si CSV no usa ; como delimitador
    df = pd.read_csv(
        "proyecto_limpio_4.csv",
        sep=';',
        dtype=str,         # mantener todo como texto
        encoding='utf-8'
    )

    # Concatenar NATURALEZA_PROYECTO + NOMBRE DEL PROYECTO_limpio
    df['TITULO_PROYECTO_MODEL'] = (
        df['NATURALEZA_PROYECTO'].fillna('') + ' ' +
        df['NOMBRE DEL PROYECTO_limpio'].fillna('')
    ).str.strip()

    resultados_json = []

    print("Ejecutando el modelo fila por fila (saltando ID = 0)...")

    for idx, fila in df.iterrows():
        id_val = str(fila.get('ID', '0')).strip()

        # Si ID es 0, no se llama al modelo; se deja respuesta vacía
        if id_val == '0':
            resultados_json.append("")
            continue

        titulo_proyecto = fila['TITULO_PROYECTO_MODEL']
        if not isinstance(titulo_proyecto, str) or not titulo_proyecto.strip():
            # Si el título concatenado está vacío, también dejamos vacío
            resultados_json.append("")
            continue

        prompt = construir_prompt(titulo_proyecto)
        respuesta_json_str = ejecutar_prompt(prompt)
        resultados_json.append(respuesta_json_str)

        # Mensaje simple de seguimiento
        print(f"Fila {idx}: procesada (ID={id_val}).")

    # Agregar columna con la respuesta JSON
    df['JSON_RPTA'] = resultados_json

    # Exportar resultados
    salida = "resultados1.csv"
    df.to_csv(salida, sep=';', index=False, encoding='utf-8-sig')
    print(f"\nProceso completado. Archivo exportado como '{salida}'.")


=== Clasificador de proyectos públicos (Gemini) ===
Leyendo archivo 'proyecto_limpio_4.csv'...
Ejecutando el modelo fila por fila (saltando ID = 0)...
Fila 0: procesada (ID=5).
Fila 1: procesada (ID=5).
Fila 2: procesada (ID=5).
Fila 3: procesada (ID=5).
Fila 5: procesada (ID=5).
Fila 6: procesada (ID=4).
Fila 7: procesada (ID=2).
Fila 8: procesada (ID=2).
Fila 10: procesada (ID=2).
Fila 11: procesada (ID=2).
Fila 12: procesada (ID=5).
Fila 13: procesada (ID=5).
Fila 15: procesada (ID=5).
Fila 16: procesada (ID=4).
Fila 18: procesada (ID=5).
Fila 19: procesada (ID=5).
Fila 20: procesada (ID=4).
Fila 21: procesada (ID=5).
Fila 22: procesada (ID=4).
Fila 23: procesada (ID=4).
Fila 24: procesada (ID=2).
Fila 25: procesada (ID=2).
Fila 28: procesada (ID=5).
Fila 30: procesada (ID=5).
Fila 31: procesada (ID=5).
Fila 32: procesada (ID=5).
Fila 34: procesada (ID=2).
Fila 36: procesada (ID=2).
Fila 37: procesada (ID=2).
Fila 39: procesada (ID=1).
Fila 40: procesada (ID=2).
Fila 41: procesada (

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 1441.11ms


Fila 182: procesada (ID=2).
Fila 183: procesada (ID=5).
Fila 184: procesada (ID=2).
Fila 185: procesada (ID=1).
Fila 187: procesada (ID=5).
Fila 188: procesada (ID=1).
Fila 189: procesada (ID=2).
Fila 190: procesada (ID=5).
Fila 191: procesada (ID=5).
Fila 192: procesada (ID=5).
Fila 193: procesada (ID=5).
Fila 194: procesada (ID=5).
Fila 198: procesada (ID=2).
Fila 199: procesada (ID=1).
Fila 200: procesada (ID=5).
Fila 202: procesada (ID=4).
Fila 203: procesada (ID=5).
Fila 207: procesada (ID=4).
Fila 209: procesada (ID=5).
Fila 210: procesada (ID=6).
Fila 211: procesada (ID=1).
Fila 212: procesada (ID=5).
Fila 213: procesada (ID=2).
Fila 214: procesada (ID=2).
Fila 215: procesada (ID=1).
Fila 216: procesada (ID=1).
Fila 217: procesada (ID=2).
Fila 218: procesada (ID=1).
Fila 219: procesada (ID=2).
Fila 220: procesada (ID=1).
Fila 221: procesada (ID=2).
Fila 222: procesada (ID=5).
Fila 223: procesada (ID=2).
Fila 224: procesada (ID=5).
Fila 226: procesada (ID=5).
Fila 227: procesada 